In [5]:
import os
import keras as K
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import LeakyReLU
from keras.utils import np_utils
from keras.optimizers import SGD, RMSprop, Adam
from keras.wrappers.scikit_learn import KerasClassifier
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

In [8]:
file = 'dataset_1'

train_datagen = K.preprocessing.image.ImageDataGenerator()

validation_datagen = K.preprocessing.image.ImageDataGenerator()

test_datagen = K.preprocessing.image.ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        os.path.join(file, 'train'),
        target_size=(50, 50),
        batch_size=1,
        color_mode='grayscale',
        class_mode='categorical',
        )

validation_generator = validation_datagen.flow_from_directory(
        os.path.join(file, 'validation'),
        target_size=(50, 50),
        batch_size=1,
        color_mode='grayscale',
        class_mode="categorical",
        )

test_generator = validation_datagen.flow_from_directory(
        os.path.join(file, 'test'),
        target_size=(50, 50),
        batch_size=1,
        color_mode='grayscale',
        class_mode="categorical",
        )

Found 12556 images belonging to 28 classes.
Found 1710 images belonging to 28 classes.
Found 3515 images belonging to 28 classes.


In [9]:
def runGenerator(generator):
    X = []
    y = []
    for i in range(len(generator)):
        X_ex, y_ex = generator.next()
        X.append(X_ex)
        y.append(y_ex)
    return X, y

def F1ScoreCalculator(test_generatorm, model):
    X_test = []
    y_test = []
    y_pred = []
    for i in range(len(test_generator)):
        X_ex, y_ex = test_generator.next()
        X_test.append(X_ex)
        y_test.append(y_ex.argmax(axis=-1))
        y_pred.append(model.predict(X_ex).argmax(axis=-1))

    return f1_score(y_test, y_pred, average='micro')

In [33]:
def Create_model(kernel_sizes=5, neurons_bk1=20, neurons_bk2=50, learn_rate=0.001, optomizer='sgd', activation='relu'):
    print('kernel_sizes= {} | neurons_bk1= {} | neurons_bk2= {} | learn_rate={} | optomizer={} | activation={}'.format(
        kernel_sizes, 
        neurons_bk1, 
        neurons_bk2, 
        learn_rate,
        optomizer,
        activation))
    
    model = Sequential()

    model.add(Conv2D(neurons_bk1, kernel_size=kernel_sizes, padding="same", input_shape=(50, 50, 1)))
    if(activation == 'relu'):
        model.add(Activation('relu'))
    else:
        model.add(LeakyReLU())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Conv2D(neurons_bk2, kernel_size=kernel_sizes, border_mode="same"))
    if(activation == 'relu'):
        model.add(Activation('relu'))
    else:
        model.add(LeakyReLU())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Flatten())
    model.add(Dense(500))
    if(activation == 'relu'):
        model.add(Activation('relu'))
    else:
        model.add(LeakyReLU())

    model.add(Dense(28))
    model.add(Activation("softmax"))
    
#     model.summary()
    if (opt == 'sgd'):
        model.compile(loss="categorical_crossentropy", optimizer=SGD(lr=learn_rate), metrics=["accuracy"])
    else:
        model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=learn_rate), metrics=["accuracy"])
    return model

In [18]:
early_stop = K.callbacks.EarlyStopping(monitor='val_acc',
                              min_delta=0,
                              patience=3,
                              verbose=0, mode='max')

In [29]:

kernel_sizes=[3, 5]
neurons_bk1=[10, 20, 30]
neurons_bk2=[50, 100]
learn_rate=[0.01, 0.005]
optomizer=['sgd', 'adam']
activation=['relu', 'leakrelu']



In [14]:
INPUT_SHAPE = (50, 50, 1)
NB_EPOCH = 20
BATCH_SIZE = 32
VERBOSE = 1
OPTIMIZER = SGD()
CLASSES = 28
STEPS = (int)(train_generator.n/BATCH_SIZE)

In [36]:
max_f1 = 0
for ks in kernel_sizes:
    for n_bk1 in neurons_bk1:
        for n_bk2 in  neurons_bk2:
            for lr in learn_rate:
                for opt in optomizer:
#                     for act in activation:
                        model = Create_model(kernel_sizes=ks,
                                            neurons_bk1=n_bk1,
                                            neurons_bk2=n_bk2,
                                            learn_rate=lr,
                                            optomizer=opt,
#                                             activation=act
                                            )
                        history = model.fit_generator(
                            train_generator,
                            steps_per_epoch=STEPS,
                            epochs=NB_EPOCH,
                            verbose=VERBOSE,
                            validation_data=validation_generator,
                    #         validation_steps=STEPS,
                            callbacks = [early_stop]
                        )
                        f1_score_val = F1ScoreCalculator(test_generator, model)
                        print ("F1-Score = {}".format(f1_score_val))
                        if (f1_score_val > max_f1):
                            best_model = model
                            max_f1 = f1_score_val
                            printer = 'kernel_sizes= {} | neurons_bk1= {} | neurons_bk2= {} | learn_rate={} | optomizer={} | activation={}'.format(
                                    kernel_sizes, 
                                    neurons_bk1, 
                                    neurons_bk2, 
                                    learn_rate,
                                    optomizer,
                                    activation)
                            print ('Best F1-Score')
                
                        print ('------ END MODEL ------\n\n')

# print(printer)

kernel_sizes= 3 | neurons_bk1= 10 | neurons_bk2= 50 | learn_rate=0.01 | optomizer=sgd | activation=relu


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, kernel_size=3, padding="same")`


Epoch 1/20


ResourceExhaustedError: OOM when allocating tensor of shape [] and type int64
	 [[Node: SGD_19/iterations/initial_value = Const[dtype=DT_INT64, value=Tensor<type: int64 shape: [] values: 0>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'SGD_19/iterations/initial_value', defined at:
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 478, in start
    self.io_loop.start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(Z

In [ ]:
history = grid.fit(X_train, y_train)

In [ ]:
print ("F1-Score = {}".format(F1ScoreCalculator(test_generator, grid)))

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model = LeNetLeakyRelu.build(input_shape=INPUT_SHAPE, classes=CLASSES)

model.summary()

model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER, metrics=["accuracy"])

In [ ]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=STEPS,
        epochs=NB_EPOCH,
#         verbose=VERBOSE,
        validation_data=validation_generator,
#         validation_steps=STEPS,
        callbacks = [early_stop]
)

In [ ]:
# pred = model.predict_generator(test_generator)
X_test = []
y_test = []
y_pred = []
for i in range(len(test_generator)):
    X_ex, y_ex = test_generator.next()
    X_test.append(X_ex)
    y_test.append(y_ex.argmax(axis=-1))
    y_pred.append(model.predict(X_ex).argmax(axis=-1))
    
val_f1_score = f1_score(y_test, y_pred, average='micro')
val_f1_score

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()